In [498]:
import pandas as pd
import gzip

In [499]:
import numpy as np
import re

Importing the dataset

In [672]:
data_path = '/Users/selima/Desktop/yt_metadata_en.jsonl.gz'

# Définir la taille de chaque chunk
chunk_size = 100000  # La taille de base de vos chunks

# Définir la quantité de chunks à lire
chunks_to_read = 3

# Initialiser une liste pour stocker les morceaux filtrés
filtered_chunk_list = []

# Utiliser gzip.open pour lire le fichier compressé
with gzip.open(data_path, 'rt', encoding='utf-8') as file:
    # Lire le fichier par morceaux avec un enumerate pour suivre le numéro du chunk
    for i, chunk in enumerate(pd.read_json(file, lines=True, chunksize=chunk_size, convert_dates=['upload_date'])):
        # Vérifier si le chunk actuel est dans la limite des chunks à lire
        if i < chunks_to_read:
            # Filtrer pour la catégorie 'Gaming' et pour les vidéos post 2016
            filtered_chunk = chunk[
                (chunk['categories'] == 'Gaming') & 
                (chunk['upload_date'] > pd.Timestamp('2016-01-01'))
            ]
            filtered_chunk_list.append(filtered_chunk)
        else:
            break  # Sortir de la boucle une fois que le nombre de chunks souhaité est atteint

# Concaténer tous les DataFrame filtrés en un seul DataFrame
final_df = pd.concat(filtered_chunk_list, ignore_index=True)

# Maintenant, final_df contient les données des trois premiers chunks filtrés selon vos critères


In [673]:
display(final_df)

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
0,Gaming,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:25.790269,Lego City Lego Police1 Hour Long Movie Cartoons about Lego City Undercover with a nice long movie to keep the kids entertained with the Lego City characters. thanks for watching!,0.0,y5IvyZlzELs,3442,9.0,"lego city,lego police,lego city police,lego fire truck,lego police chase,lego fireman,lego firetruck,lego movies,policias lego,lego polizi,lego police robbers,lego city undercover,lego clowns",Lego City Lego Police1 Hour Long Movie Cartoons about Lego City Undercover,2016-09-26,1253
1,Gaming,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:28.334071,Lego City Police Lego Fireman Firetruck Long Movie Lego City Undercover for kids with a nice long video for the kids to enjoy and learn while having fun. Thanks for watching!,0.0,m1agc0qT0BY,2407,8.0,"lego city,lego police,lego city police,lego fire truck,lego cartoons,lego undercover,lego city movie,videos de lego city,lego bomberos,lego policias,lego polizi,lego movie,lego episodes",Lego City Police Lego Fireman Firetruck Long Movie Lego City Undercover for kids,2016-09-25,2311
2,Gaming,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:32.174979,Lego Dimensions Cartoons Movie Lego Cartoons Lego Batman Lego Movie for kids with all the adventures from the characters of Lego Dimensions and Lego City with Batman Hobbit and many more\nThanks for watching!,1.0,rr6tfbBA9iY,1820,11.0,"lego city,lego dimensions,lego batman,lego dimensions ps4,lego dimensions movie,lego movies,cartoons about lego,lego cartoons,lego for kids,lego movie for kids,lego dimension",Lego Dimensions Cartoons Movie Lego Cartoons Lego Batman Lego City Movie for kids,2016-09-24,5596
3,Gaming,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:25.379061,"Lego City Police Lego Fireman Firetruck Cartoons about Lego City Movie for kids Episode with a great long video to keep the little ones entertained while learning and having fun with the Lego City characters that the kids love. Thanks for watching, please Like!",0.0,ZGll5_wD9Ys,1209,8.0,"lego city,lego city police,lego fire truck,lego fireman,bomberos paraniños,policias para niños,lego bomberos,lego polizi,lego policias,lego movies,lego cartoons,lego police car,lego car chase,lego city undercover",Lego City Police Lego Fireman Firetruck Cartoons about Lego City Movie for kids Episodes,2016-09-21,792
4,Gaming,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:26.020550,"Lego Jurassic World Complete Movie for kids Lego dinosaurs with a nice long video to keep the little ones entertained while learning and having fun with the Lego characters from Lego City . Thanks for watching, please Like! Лего фильм",426.0,kYkokQgnu20,2053,2076.0,"lego jurassic world,lego city,lego city for kids,lego dinosaurs,lego movies,lego cartoons,pelicula lego,lego pelicula,lego español,Лего фильм,lego movies for kids,lego films,lego dinosaurios",Lego Jurassic World Complete Movie for kids Lego dinosaurs Лего фильм,2016-09-21,1141393
...,...,...,...,...,...,...,...,...,...,...,...,...
44127,Gaming,UCzFUs-fmDxTSIoaWYeFFyOA,2019-11-02 21:13:49.815566,"(Subtitulos en español añadidos!)\n\nWatch the rest of the series first!\n----------------\n1- Megalomaniac: https://youtu.be/BCzDrlnOYY0\n2- Yet Darker: https://youtu.be/l4tGTxbnujY\n3- *Determination.: https://youtu.be/F-iO-I0oGn8\n4- Your Best Friend: https://youtu.be/3_kc80dPD5g\n5- Continue: (You're watching it)\n---------------------------------------------\n\nGlitchtale Season 1 Finale!\n\nOh boi, this was quite a ride. After almost 2 months of work I finally finished!. Nyx The Shield made the soundtrack and Strelok added the sound effects, I'm lucky to have them as my friends honestly. I hope you all enjoy this season finale, get hyped for season 2!. Well not too hype, I will take a small break after all. However I will be streaming some gaming in the meantime, see ya on my twitch!.\n\nChara 

In [674]:
data = final_df.copy()

In [675]:
data2 = data.copy()

* We will work on identifying collaboration

First, we will see if there is any of this pattern of links in the description

identified channel links types:
1) youtube.com/channel/youtubecreators
2) youtube.com/@youtubecreators
3) youtube.com/c/YouTubeCreators
4) youtube.com/user/YouTube


We know that for URL 1) and 4) we can find the channel ID wheras in URL 2) and 3) we will find the channel names

So, we write a function to scap all the youtube links in the description, then if it is a name of a channel, we append it to a new column in our dataframe, and if it is a id we append it to another one.

If there is more than a channel mentionned, we take all the names.

In [681]:
# function to extract all youtube channel IDs and channel names
#4.4 seconds to run
def extract_youtube_channels(text):
    # Regular expressions for finding YouTube channel URLs
    channel_id_pattern = r'https?://www\.youtube\.com/(?:channel/|user/)([\w-]+)'
    channel_name_pattern = r'https?://www\.youtube\.com/(?:@|c/)([\w-]+)'

    # Find all links that match the patterns
    channel_ids = re.findall(channel_id_pattern, text)
    channel_names = re.findall(channel_name_pattern, text)

    # Join the IDs and names found (or np.nan if not found)
    channel_id = ', '.join(channel_ids) if channel_ids else pd.NA
    channel_name = ', '.join(channel_names) if channel_names else pd.NA

    return channel_id, channel_name

# apply the function and create two new columns
data2[['mentionned_channel_ID', 'mentionned_channel_name']] = data2['description'].apply(
    lambda x: pd.Series(extract_youtube_channels(x))
)



In [554]:
data2[['channel_id' , 'description', 'mentionned_channel_ID', 'mentionned_channel_name']].isnull().sum()

channel_id                     0
description                    0
mentionned_channel_ID      34245
mentionned_channel_name    41839
dtype: int64

In [555]:
data2.shape[0]

44132

We don't need the description, title and tags anymore, let's work on a less heavy dataset

In [556]:
data2.drop(['tags', 'description' , 'title'] , axis = 1, inplace= True)

We will now add the dataset (df_channels_en.tsv.gz) where we have both the names and the IDS of the channels. This is to have a consistent dataset where we will have either the names of the channels or the IDs.

 As in the dataset we will use after (df_timeseries_en.csv.gz) there are only the ids of the channels, we will consider the IDs.

In [557]:
df_channels_path = '/Users/selima/Downloads/df_channels_en.tsv'
df_channels = pd.read_csv(df_channels_path, sep ='\t')

In [558]:
df_channels.columns

Index(['category_cc', 'join_date', 'channel', 'name_cc', 'subscribers_cc',
       'videos_cc', 'subscriber_rank_sb', 'weights'],
      dtype='object')

Keeping only the channel names and IDs.

In [559]:
#channel_ids_and_names = df_channels[df_channels['category_cc'] == 'Gaming'][['channel' , 'name_cc']].copy()
#renaming the column for it to be readble 
channel_ids_and_names = df_channels[['channel' , 'name_cc']].copy()
channel_ids_and_names.rename(columns={"channel": "channel_id", "name_cc": "channel_name"} , inplace= True)

How we will use this dataset?

Step 1) 

  a)  If one of the IDS in the column mentionned_channel_ID doesn't appear in this dataset, we take it off because we can't work on it.
  b) If a youtuber is mentionning his own channel , we take the ID off


Step 2) Find the ids of the mentionned_channel_name and append them , to be consistent and have only ids (because in our other dataset we have the ids)

Working on Step 1)

In [560]:
unique_channel_ids = channel_ids_and_names.channel_id.unique()
#convert to a set for faster loop
unique_ids_set = set(unique_channel_ids)


In [682]:
def filter_ids_by_existance_and_not_self_mentioning(df, unique_ids_set):
    # Define a vectorized function
    def process_entry(mentioned_ids, channel_id):
        if pd.isna(mentioned_ids):
            return pd.NA
        ids = [id.strip() for id in mentioned_ids.split(',')]
        valid_ids = [id for id in ids if id in unique_ids_set and id != channel_id]
        return ','.join(valid_ids) if valid_ids else pd.NA

    # Split, process, and rejoin without using apply row-wise
    filtered = [process_entry(m_id, c_id) for m_id, c_id in zip(df['mentionned_channel_ID'], df['channel_id'])]
    
    return filtered

In [683]:
data2['filtered_mentionned_channel_ID'] = filter_ids_by_existance_and_not_self_mentioning(data2[['mentionned_channel_ID' ,
                                                                                                   'channel_id']], unique_ids_set)

In [684]:
data2[['channel_id' , 'filtered_mentionned_channel_ID' ]].dropna().head(20)

,channel_id,filtered_mentionned_channel_ID
926,UCzWm1-4XF7AHxVUTkHCM1uw,UC_0CVCfC_3iuHqmyClu59Uw
1619,UCzWfqsjlvzAOUozsGFR3kbg,UC3S6nIDGJ5OtpC-mbvFA8Ew
1847,UCzWLQNPQ3m_JyxIeNFyTrsg,UC_QHwXH3BX4M2qxnGquXJTw
1860,UCzWLQNPQ3m_JyxIeNFyTrsg,UCPqI8burrYQiGSZgBw0qFdQ
1875,UCzWLQNPQ3m_JyxIeNFyTrsg,UCMxuiOifJ6WAp-KfgnbNLxg
2003,UCzWEqovJ7CvRYBLJ5D7-rFA,UCXWEa9Rl4LPMG0dygqnVCQg
2215,UCzVnXCLWqY5dtOBn9UZoFTw,"UCOWAOUxf3XILuywxw60g5vA,UC-P2nvnAm3kUQH43M_6GpDQ"
2216,UCzVnXCLWqY5dtOBn9UZoFTw,"UCOWAOUxf3XILuywxw60g5vA,UC-P2nvnAm3kUQH43M_6GpDQ"
2217,UCzVnXCLWqY5dtOBn9UZoFTw,"UCOWAOUxf3XILuywxw60g5vA,UC-P2nvnAm3kUQH43M_6GpDQ"
2220,UCzVnXCLWqY5dtOBn9UZoFTw,"UCOWAOUxf3XILuywxw60g5vA,UC-P2nvnAm3kUQH43M_6GpDQ"


In [685]:
data2.filtered_mentionned_channel_ID.dropna().shape[0]

1675

OK, Now we have a new column nammed 'filtered_mentionned_channel_ID' .


 We are sure that within this column, we only have valid channel ids (ones that exists in the dataset df_channels_en.tsv.gz )

 and we are sure it is not a youtuber mentionning himself

Working on step 2)

Having the column 'mentionned_channel_name' , 

1) Find the channel_ids of the mentionned_channel_name and append them, to be consistent and have only ids (because in our other dataset we have the ids) --> a) check that the names are valid (exist in our dataframe) b) add the IDs corresponding to valid names

2) We need to check that a) the channel_name isn't the name of the video creator, b) the id of the channel name doesn't appear in our filtered_mentionned_channel_ID


In [686]:
# Create a dictionary for faster lookup
channel_name_to_id = pd.Series(channel_ids_and_names.channel_id.values, index=channel_ids_and_names.channel_name.str.lower()).to_dict()

In [689]:
def get_channel_ids_from_valid_names(mentionned_named, channel_name_to_id):
    def process_entry(entry):
        # Handle missing values
        if pd.isna(entry):
            return pd.NA
        names = str(entry).split(',')
        # Lookup each name in the channel_name_to_id dictionary
        valid_ids = [channel_name_to_id.get(name.strip().lower()) for name in names if name.strip().lower() in channel_name_to_id]
        # Filter out None values in case a name wasn't found in the dictionary
        valid_ids = [id for id in valid_ids if id is not None]
        return ','.join(valid_ids) if valid_ids else pd.NA

    # list comprehension instead of apply for potentially better performance
    return [process_entry(entry) for entry in mentionned_named]



In [690]:
data2['ids_from_valid_names'] = get_channel_ids_from_valid_names(data2['mentionned_channel_name'], channel_name_to_id)

In [691]:
len(data2['ids_from_valid_names'].dropna())

133

In [692]:
data2[['channel_id' , 'filtered_mentionned_channel_ID' , 'mentionned_channel_name' , 'ids_from_valid_names' ]].dropna()

,channel_id,filtered_mentionned_channel_ID,mentionned_channel_name,ids_from_valid_names
2442,UCzV74pBOfwtAMuOm-Jg8gqg,"UC_QHwXH3BX4M2qxnGquXJTw,UCqkjQ4yDB9MeajgHQGLHPmg","flanke, pikachuplays, RKYmc",UCJs2lYkiApLerCx1VqZyi7Q
2443,UCzV74pBOfwtAMuOm-Jg8gqg,"UC_QHwXH3BX4M2qxnGquXJTw,UCqkjQ4yDB9MeajgHQGLHPmg","flanke, pikachuplays, RKYmc",UCJs2lYkiApLerCx1VqZyi7Q
2444,UCzV74pBOfwtAMuOm-Jg8gqg,"UC_QHwXH3BX4M2qxnGquXJTw,UCqkjQ4yDB9MeajgHQGLHPmg","flanke, pikachuplays, RKYmc",UCJs2lYkiApLerCx1VqZyi7Q
2545,UCzV74pBOfwtAMuOm-Jg8gqg,"UCzMjRlKVO9XIqH_crIFpi6w,UCqkjQ4yDB9MeajgHQGLHPmg","shutupbrick, Spifey","UCjOUBkgXwhurLFYfEJAfVKQ,UCaawTX2yTGhd3xKwKfpXZTQ"
2547,UCzV74pBOfwtAMuOm-Jg8gqg,"UCzMjRlKVO9XIqH_crIFpi6w,UCqkjQ4yDB9MeajgHQGLHPmg","shutupbrick, Spifey","UCjOUBkgXwhurLFYfEJAfVKQ,UCaawTX2yTGhd3xKwKfpXZTQ"
...,...,...,...,...
12760,UCzPKqMawrz7ov21YBEXAZzg,"UCeE4VazTtIl502GDwUndZCQ,UChN_oPdgS6kWyQ-usdeN9fg",kaykrae,UCOds3oNaKqV-a4XQrK6K7Qg
12762,UCzPKqMawrz7ov21YBEXAZzg,"UCeE4VazTtIl502GDwUndZCQ,UChN_oPdgS6kWyQ-usdeN9fg",kaykrae,UCOds3oNaKqV-a4XQrK6K7Qg
12764,UCzPKqMawrz7ov21YBEXAZzg,"UCeE4VazTtIl502GDwUndZCQ,UChN_oPdgS6kWyQ-usdeN9fg",kaykrae,UCOds3oNaKqV-a4XQrK6K7Qg
12765,UCzPKqMawrz7ov21YBEXAZzg,"UCeE4VazTtIl502GDwUndZCQ,UChN_oPdgS6kWyQ-usdeN9fg",kaykrae,UCOds3oNaKqV-a4XQrK6K7Qg


Now : check that the id we got isn't 1) of the video creator , 2) doesn't appear in filtered_mentionned_channel_ID

In [694]:
# Step 2: Remove redundant IDs
# a) Ensure ids_from_valid_names are not the same as the channel_id
# b) Ensure ids_from_valid_names do not appear in filtered_mentionned_channel_ID

def remove_redundant_ids(row):
    # Split the ids into a list, remove any whitespace, and filter out any empty strings
    valid_ids = list(filter(None, [x.strip() for x in str(row['ids_from_valid_names']).split(',')]))
    mentioned_ids = set(filter(None, [x.strip() for x in str(row['filtered_mentionned_channel_ID']).split(',')]))
    
    # Remove the channel_id from the valid_ids if it's present
    valid_ids = [id for id in valid_ids if id != row['channel_id']]
    
    # Remove any id from valid_ids if it's already present in mentioned_ids
    valid_ids = [id for id in valid_ids if id not in mentioned_ids]
    
    # Join the valid ids back into a string
    return ','.join(valid_ids) if valid_ids else pd.NA

# Apply the function to each row of the dataframe
data2['clean_ids_from_valid_names'] = data2.apply(remove_redundant_ids, axis=1)



In [695]:
data2['clean_ids_from_valid_names']
data2[['channel_id' , 'filtered_mentionned_channel_ID' , 'mentionned_channel_name' , 'ids_from_valid_names' , 'clean_ids_from_valid_names']].dropna()

,channel_id,filtered_mentionned_channel_ID,mentionned_channel_name,ids_from_valid_names,clean_ids_from_valid_names
2442,UCzV74pBOfwtAMuOm-Jg8gqg,"UC_QHwXH3BX4M2qxnGquXJTw,UCqkjQ4yDB9MeajgHQGLHPmg","flanke, pikachuplays, RKYmc",UCJs2lYkiApLerCx1VqZyi7Q,UCJs2lYkiApLerCx1VqZyi7Q
2443,UCzV74pBOfwtAMuOm-Jg8gqg,"UC_QHwXH3BX4M2qxnGquXJTw,UCqkjQ4yDB9MeajgHQGLHPmg","flanke, pikachuplays, RKYmc",UCJs2lYkiApLerCx1VqZyi7Q,UCJs2lYkiApLerCx1VqZyi7Q
2444,UCzV74pBOfwtAMuOm-Jg8gqg,"UC_QHwXH3BX4M2qxnGquXJTw,UCqkjQ4yDB9MeajgHQGLHPmg","flanke, pikachuplays, RKYmc",UCJs2lYkiApLerCx1VqZyi7Q,UCJs2lYkiApLerCx1VqZyi7Q
2545,UCzV74pBOfwtAMuOm-Jg8gqg,"UCzMjRlKVO9XIqH_crIFpi6w,UCqkjQ4yDB9MeajgHQGLHPmg","shutupbrick, Spifey","UCjOUBkgXwhurLFYfEJAfVKQ,UCaawTX2yTGhd3xKwKfpXZTQ","UCjOUBkgXwhurLFYfEJAfVKQ,UCaawTX2yTGhd3xKwKfpXZTQ"
2547,UCzV74pBOfwtAMuOm-Jg8gqg,"UCzMjRlKVO9XIqH_crIFpi6w,UCqkjQ4yDB9MeajgHQGLHPmg","shutupbrick, Spifey","UCjOUBkgXwhurLFYfEJAfVKQ,UCaawTX2yTGhd3xKwKfpXZTQ","UCjOUBkgXwhurLFYfEJAfVKQ,UCaawTX2yTGhd3xKwKfpXZTQ"
...,...,...,...,...,...
12760,UCzPKqMawrz7ov21YBEXAZzg,"UCeE4VazTtIl502GDwUndZCQ,UChN_oPdgS6kWyQ-usdeN9fg",kaykrae,UCOds3oNaKqV-a4XQrK6K7Qg,UCOds3oNaKqV-a4XQrK6K7Qg
12762,UCzPKqMawrz7ov21YBEXAZzg,"UCeE4VazTtIl502GDwUndZCQ,UChN_oPdgS6kWyQ-usdeN9fg",kaykrae,UCOds3oNaKqV-a4XQrK6K7Qg,UCOds3oNaKqV-a4XQrK6K7Qg
12764,UCzPKqMawrz7ov21YBEXAZzg,"UCeE4VazTtIl502GDwUndZCQ,UChN_oPdgS6kWyQ-usdeN9fg",kaykrae,UCOds3oNaKqV-a4XQrK6K7Qg,UCOds3oNaKqV-a4XQrK6K7Qg
12765,UCzPKqMawrz7ov21YBEXAZzg,"UCeE4VazTtIl502GDwUndZCQ,UChN_oPdgS6kWyQ-usdeN9fg",kaykrae,UCOds3oNaKqV-a4XQrK6K7Qg,UCOds3oNaKqV-a4XQrK6K7Qg


now, concatenate filtered_mentionned_channel_ID and clean_ids_from_valid_names to have the final colaborator_ids

In [696]:
def concatenate_columns(row):
    if np.isna(row['filtered_mentionned_channel_ID']) and np.isna(row['clean_ids_from_valid_names']):
        return np.nan
    elif  np.isna(row['filtered_mentionned_channel_ID']):
        return row['clean_ids_from_valid_names']
    elif np.isna(row['clean_ids_from_valid_names']):
        return row['filtered_mentionned_channel_ID']
    else:
        return f"{row['filtered_mentionned_channel_ID']},{row['clean_ids_from_valid_names']}"

In [697]:
# Define a function to handle the concatenation and NaN logic
def concatenate_columns(row):
    val1 = row['filtered_mentionned_channel_ID']
    val2 = row['clean_ids_from_valid_names']
    
    # Check for both np.nan and '<NA>' string
    if (pd.isna(val1) or val1 == '<NA>') and (pd.isna(val2) or val2 == '<NA>'):
        return np.nan
    elif pd.isna(val1) or val1 == '<NA>':
        return val2
    elif pd.isna(val2) or val2 == '<NA>':
        return val1
    else:
        return f"{val1},{val2}"

# Replace '<NA>' with np.nan for consistency
data2.replace('<NA>', np.nan, inplace=True)

# Apply this function across the rows
data2['final_colaborator_ids'] = data2.apply(concatenate_columns, axis=1)


In [698]:
display(data2[['channel_id' ,'final_colaborator_ids']].dropna().head(30))

,channel_id,final_colaborator_ids
926,UCzWm1-4XF7AHxVUTkHCM1uw,UC_0CVCfC_3iuHqmyClu59Uw
1619,UCzWfqsjlvzAOUozsGFR3kbg,UC3S6nIDGJ5OtpC-mbvFA8Ew
1847,UCzWLQNPQ3m_JyxIeNFyTrsg,UC_QHwXH3BX4M2qxnGquXJTw
1860,UCzWLQNPQ3m_JyxIeNFyTrsg,UCPqI8burrYQiGSZgBw0qFdQ
1875,UCzWLQNPQ3m_JyxIeNFyTrsg,UCMxuiOifJ6WAp-KfgnbNLxg
2003,UCzWEqovJ7CvRYBLJ5D7-rFA,UCXWEa9Rl4LPMG0dygqnVCQg
2215,UCzVnXCLWqY5dtOBn9UZoFTw,"UCOWAOUxf3XILuywxw60g5vA,UC-P2nvnAm3kUQH43M_6GpDQ"
2216,UCzVnXCLWqY5dtOBn9UZoFTw,"UCOWAOUxf3XILuywxw60g5vA,UC-P2nvnAm3kUQH43M_6GpDQ"
2217,UCzVnXCLWqY5dtOBn9UZoFTw,"UCOWAOUxf3XILuywxw60g5vA,UC-P2nvnAm3kUQH43M_6GpDQ"
2220,UCzVnXCLWqY5dtOBn9UZoFTw,"UCOWAOUxf3XILuywxw60g5vA,UC-P2nvnAm3kUQH43M_6GpDQ"
